In [66]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_validate
import pandas as pd

In [67]:
dataset = pd.read_csv('50_Startups.csv')

In [68]:
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [69]:
# DATSET INFO -> Dataset to observe if there are any correlations between profit and expenditure across  verticals ( R&D Spend,Adminitsration Spend, Marketing Spend ) & also across the vertical in which the state operates

In [70]:
# To Model eqn in form : y = a + bx + cz + du , State is our categorical variable

In [71]:
# We should LableL Encode State to form dummy variables. However, be sure to avoid the dummy variable trap i.e. to include all our dummy variables in the model.
# By Rule of thumb, always be sure to remove atleast one dummy varaible as one of them is already represented in 'a', refer to eqn above

In [72]:
# Concept : P- Value
# https://www.mathbootcamps.com/what-is-a-p-value/
# https://www.wikihow.com/Calculate-P-Value

In [73]:
dataset.describe()

,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200
std,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,51283.140000,0.000000,14681.400000
25%,39936.370000,103730.875000,129300.132500,90138.902500
50%,73051.080000,122699.795000,212716.240000,107978.190000
75%,101602.800000,144842.180000,299469.085000,139765.977500
max,165349.200000,182645.560000,471784.100000,192261.830000


In [74]:
x = dataset.iloc[:,:-1].values
x[:5,:]

array([[165349.2, 136897.8, 471784.1, 'New York'],
       [162597.7, 151377.59, 443898.53, 'California'],
       [153441.51, 101145.55, 407934.54, 'Florida'],
       [144372.41, 118671.85, 383199.62, 'New York'],
       [142107.34, 91391.77, 366168.42, 'Florida']], dtype=object)

In [75]:
y = dataset.loc[:,['Profit']].values
y[:5]

array([[192261.83],
       [191792.06],
       [191050.39],
       [182901.99],
       [166187.94]])

In [76]:
# Encode the State Column in X
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelEncoder_x = LabelEncoder()
x[:,3] = labelEncoder_x.fit_transform(x[:,3])
oneHotEncoder_x = OneHotEncoder(categorical_features=[3])
# categorical_features -> Inde of column we want to one hot encode
x = oneHotEncoder_x.fit_transform(x).toarray()

C:\Users\rangan_basu\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\rangan_basu\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:450: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [77]:
x[:10,:].astype(int) 
# Only to view for our convinience

array([[     0,      0,      1, 165349, 136897, 471784],
       [     1,      0,      0, 162597, 151377, 443898],
       [     0,      1,      0, 153441, 101145, 407934],
       [     0,      0,      1, 144372, 118671, 383199],
       [     0,      1,      0, 142107,  91391, 366168],
       [     0,      0,      1, 131876,  99814, 362861],
       [     1,      0,      0, 134615, 147198, 127716],
       [     0,      1,      0, 130298, 145530, 323876],
       [     0,      0,      1, 120542, 148718, 311613],
       [     1,      0,      0, 123334, 108679, 304981]])

In [78]:
# Avoiding the dummy Variable Trap
x = x[:,1:]
# Not required as sk-learn already takes care of this for us, but still done now just for demo

In [79]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [80]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(xtrain,ytrain)
y_pred = model.predict(xtest)
y_pred

array([[126362.87908252],
       [ 84608.45383643],
       [ 99677.49425155],
       [ 46357.46068582],
       [128750.48288497],
       [ 50912.41741905],
       [109741.350327  ],
       [100643.24281644],
       [ 97599.275746  ],
       [113097.42524437]])

In [81]:
ytest

array([[134307.35],
       [ 81005.76],
       [ 99937.59],
       [ 64926.08],
       [125370.37],
       [ 35673.41],
       [105733.54],
       [107404.34],
       [ 97427.84],
       [122776.86]])

In [82]:
# Building the optimal model using Backward Elimination
import statsmodels.formula.api as sm
# statsmodel doesnot include constant b(0) in its prediction for y = b(0) + b(1)x(1)+ b(2)x(2)....+b(n)x(n),but b(0) is accounted for in skLearn Linear Regression Model
# Hence we add a column of ones to account for the above scenario essentially making the eqn y = b(0)*1 = .......

In [83]:
x = np.append(arr = np.ones((50,1)).astype(int),values=x,axis=1)
# to a vector of 50 rows and one column having 1 append x 
# axis = 1 or (column)
x[:10,:].astype(int)

array([[     1,      0,      1, 165349, 136897, 471784],
       [     1,      0,      0, 162597, 151377, 443898],
       [     1,      1,      0, 153441, 101145, 407934],
       [     1,      0,      1, 144372, 118671, 383199],
       [     1,      1,      0, 142107,  91391, 366168],
       [     1,      0,      1, 131876,  99814, 362861],
       [     1,      0,      0, 134615, 147198, 127716],
       [     1,      1,      0, 130298, 145530, 323876],
       [     1,      0,      1, 120542, 148718, 311613],
       [     1,      0,      0, 123334, 108679, 304981]])

In [84]:
# X-optimal -> Will ultimately only conatin those features which will have a high impact on profit
# Backward Elemination -> Remove one by one all variables which are not statistically significant
X_opt = x[:,[0,1,2,3,4,5]] 
# At first, we include all features and select a significance level to stay in model, i.e p>SL
# Then fit full model with all possible predictors
# Considr predictor with highest p-value
# Remove predictor
# Fit model without the variable 
# -> Repeat last 3 till p<SL

In [85]:
sm.OLS?

In [88]:
regressor_OLS = sm.OLS(endog=y,exog=X_opt).fit() # Ordinary Least Squares
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Thu, 30 May 2019   Prob (F-statistic):           1.34e-27
Time:                        02:59:12   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
x1           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
x2           -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
x3             0.8060      0.046     17.369      0.000       0.712       0.900
x4            -0.0270      0.052     -0.517      0.608      -0.132       0.078
x5             0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [89]:
# The lower the p value, the more signficant or indepent variable is w.r.t the dependent variable
# cosnt -> x(0) = 1, added by us at begining of x
# Let's say we kept significance level as 0.05
# x2 -> One of the dummy variables from One Hot Encoding has the highest p value
# Remove x2

In [90]:
# So now our X-opt becomes as below
X_opt = x[:,[0,1,3,4,5]]
regressor_OLS = sm.OLS(endog=y,exog=X_opt).fit() # Ordinary Least Squares
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Thu, 30 May 2019   Prob (F-statistic):           8.49e-29
Time:                        03:04:25   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.011e+04   6647.870      7.537      0.000    3.67e+04    6.35e+04
x1           220.1585   2900.536      0.076      0.940   -5621.821    6062.138
x2             0.8060      0.046     17.606      0.000       0.714       0.898
x3            -0.0270      0.052     -0.523      0.604      -0.131       0.077
x4             0.0270      0.017      1.592      0.118      -0.007       0.061
==============================================================================
Omnibus:                       14.758   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.172
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [91]:
# Remove X1
X_opt = x[:,[0,3,4,5]]
regressor_OLS = sm.OLS(endog=y,exog=X_opt).fit() # Ordinary Least Squares
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Thu, 30 May 2019   Prob (F-statistic):           4.53e-30
Time:                        03:04:57   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.012e+04   6572.353      7.626      0.000    3.69e+04    6.34e+04
x1             0.8057      0.045     17.846      0.000       0.715       0.897
x2            -0.0268      0.051     -0.526      0.602      -0.130       0.076
x3             0.0272      0.016      1.655      0.105      -0.006       0.060
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [95]:
# Remove X2
X_opt = x[:,[0,3,5]]
regressor_OLS = sm.OLS(endog=y,exog=X_opt).fit() # Ordinary Least Squares
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Thu, 30 May 2019   Prob (F-statistic):           2.16e-31
Time:                        03:15:51   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.000    4.16e+04    5.24e+04
x1             0.7966      0.041     19.266      0.000       0.713       0.880
x2             0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [96]:
X_opt = X_opt[:,[1,2]]

In [99]:
optimized_model =  LinearRegression()
optimized_model.fit(xtrain,ytrain)
y_pred = optimized_model.predict(xtest)
y_pred

array([[126362.87908252],
       [ 84608.45383643],
       [ 99677.49425155],
       [ 46357.46068582],
       [128750.48288497],
       [ 50912.41741905],
       [109741.350327  ],
       [100643.24281644],
       [ 97599.275746  ],
       [113097.42524437]])

In [101]:
ytest

array([[134307.35],
       [ 81005.76],
       [ 99937.59],
       [ 64926.08],
       [125370.37],
       [ 35673.41],
       [105733.54],
       [107404.34],
       [ 97427.84],
       [122776.86]])

In [105]:
# Reference Codes :
# Backward Elimination with p-values only:
import statsmodels.formula.api as sm
def backwardElimination(x, sl):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
    regressor_OLS.summary()
    return x
 
SL = 0.07
X_opt = x[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backwardElimination(X_opt, SL)

In [106]:
X_Modeled.astype(int)

array([[     1, 165349, 471784],
       [     1, 162597, 443898],
       [     1, 153441, 407934],
       [     1, 144372, 383199],
       [     1, 142107, 366168],
       [     1, 131876, 362861],
       [     1, 134615, 127716],
       [     1, 130298, 323876],
       [     1, 120542, 311613],
       [     1, 123334, 304981],
       [     1, 101913, 229160],
       [     1, 100671, 249744],
       [     1,  93863, 249839],
       [     1,  91992, 252664],
       [     1, 119943, 256512],
       [     1, 114523, 261776],
       [     1,  78013, 264346],
       [     1,  94657, 282574],
       [     1,  91749, 294919],
       [     1,  86419,      0],
       [     1,  76253, 298664],
       [     1,  78389, 299737],
       [     1,  73994, 303319],
       [     1,  67532, 304768],
       [     1,  77044, 140574],
       [     1,  64664, 137962],
       [     1,  75328, 134050],
       [     1,  72107, 353183],
       [     1,  66051, 118148],
       [     1,  65605, 107138],
       [  

In [108]:
# Backward Elimination with p-values and Adjusted R Squared:
import statsmodels.formula.api as sm
def backwardElimination(x, SL):
    numVars = len(x[0])
    temp = np.zeros((50,6)).astype(int)
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        adjR_before = regressor_OLS.rsquared_adj.astype(float)
        if maxVar > SL:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    temp[:,j] = x[:, j]
                    x = np.delete(x, j, 1)
                    tmp_regressor = sm.OLS(y, x).fit()
                    adjR_after = tmp_regressor.rsquared_adj.astype(float)
                    if (adjR_before >= adjR_after):
                        x_rollback = np.hstack((x, temp[:,[0,j]]))
                        x_rollback = np.delete(x_rollback, j, 1)
                        print (regressor_OLS.summary())
                        return x_rollback
                    else:
                        continue
    regressor_OLS.summary()
    return x
 
SL = 0.05
X_opt = x[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backwardElimination(X_opt, SL)

In [109]:
X_Modeled.astype(int)

array([[     1, 165349, 471784],
       [     1, 162597, 443898],
       [     1, 153441, 407934],
       [     1, 144372, 383199],
       [     1, 142107, 366168],
       [     1, 131876, 362861],
       [     1, 134615, 127716],
       [     1, 130298, 323876],
       [     1, 120542, 311613],
       [     1, 123334, 304981],
       [     1, 101913, 229160],
       [     1, 100671, 249744],
       [     1,  93863, 249839],
       [     1,  91992, 252664],
       [     1, 119943, 256512],
       [     1, 114523, 261776],
       [     1,  78013, 264346],
       [     1,  94657, 282574],
       [     1,  91749, 294919],
       [     1,  86419,      0],
       [     1,  76253, 298664],
       [     1,  78389, 299737],
       [     1,  73994, 303319],
       [     1,  67532, 304768],
       [     1,  77044, 140574],
       [     1,  64664, 137962],
       [     1,  75328, 134050],
       [     1,  72107, 353183],
       [     1,  66051, 118148],
       [     1,  65605, 107138],
       [  